## ⚙️ Setup:

### Set Up Virtual Environment & Install Dependencies
In terminal, run:

```
uv venv --python 3.12
source .venv/bin/activate
uv sync
```

<i>Note: dependencies are defined in pyproject.toml</i>

### Configure your API Keys

**1. Get your API keys**

If you don't have one already, 
- create [Gemini API key in Google AI Studio](https://aistudio.google.com/app/api-keys)
- create [SERP API](https://serpapi.com)

**2. Add the key to .env file in the same location as your notebook and add the below lines in it**

```
GOOGLE_API_KEY=your_google_api_key_here
SERPAPI_API_KEY=your_serp_api_key_here
```

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

try:
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    SERPAPI_API_KEY = os.environ.get("SERPAPI_API_KEY")
    print("✅ Gemini & SERP API keys setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' & SERPAPI_API_KEY to your .env file. Details: {e}")

✅ Gemini & SERP API keys setup complete.


### Import ADK components

In [ ]:
from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from pydantic import BaseModel, Field


print("✅ ADK components imported successfully.")

## 🤖🌍✈️🏨📉 AI Living Trip Optimizer & Deal Hunter:

### 🤖 Preference Collector Agent
- Captures user's preferences in a structured output

In [ ]:
class UserPreferences(BaseModel):
    destination: str = Field(description="The travel destination")
    travel_dates: str = Field(description="The travel dates")
    budget: float = Field(description="The budget for the trip")
    interests: list = Field(description="List of user interests for the trip")

In [ ]:
preference_collector_agent = Agent(
    name="PreferenceCollectorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Preference Collector Agent. Receive user input about their travel preferences.
    Store user preferences in a structured format for further processing.""",
    output_schema=UserPreferences,
    output_key="user_preferences")

print("✅ preference_collector_agent created.")

In [ ]:
# Root Agent: Its job is to start the workflow after receiving user input.
root_agent = SequentialAgent(
    name="TripOptimizerDealHunterAgent",
    sub_agents=[preference_collector_agent])

print("✅ root_agent created.")

In [ ]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner created.")
response = await runner.run_debug("Plan a trip to Bangkok from Mar 10–15, budget $1200, style: food + culture.")
print(response)